# 直接存页码csv和内容json

In [1]:
# !pip install pypdf pandas tqdm spacy rapidfuzz dateparser tzlocal PyPDF2 pymupdf pytesseract pillow
# !python -m spacy download en_core_web_sm

In [6]:
from pathlib import Path
import subprocess
import sys
import concurrent.futures
import os

In [7]:
os.cpu_count()

16

In [8]:
# Configuration
directories_to_process = [
    # "G:\\My Drive\\NUS MSBA SEM2\\<Base Folder>\\files",
    "C:\\Users\\22601\\Downloads\\downloads\\files"
]

In [9]:
script_path = Path("./parallel_pdf_processor.py").resolve()

In [10]:

def process_directory(dir_path_str):
    """Process a single directory with the PDF processor."""
    dir_path = Path(dir_path_str).resolve()
    if not dir_path.is_dir():
        print(f"Directory not found: {dir_path}")
        return -1
    
    print(f"Processing: {dir_path.name}")
    
    try:
        result = subprocess.run(
            [sys.executable, str(script_path), str(dir_path)],
            capture_output=True,
            text=True,
            timeout=3600  # 1 hour timeout
        )
        
        if result.stdout:
            print(f"[{dir_path.name}] {result.stdout}")
        if result.stderr:
            print(f"[{dir_path.name}] ERROR: {result.stderr}")
            
        return result.returncode
        
    except subprocess.TimeoutExpired:
        print(f"[{dir_path.name}] Timeout after 1 hour")
        return -2
    except Exception as e:
        print(f"[{dir_path.name}] Failed: {e}")
        return -3

if __name__ == "__main__":
    print(f"Processing {len(directories_to_process)} directories...")
    
    # Use ThreadPool for I/O-bound subprocess management
    # Max workers = min(directory count, CPU cores)
    max_workers = os.cpu_count() - 4
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(process_directory, d): d for d in directories_to_process}
        
        for future in concurrent.futures.as_completed(futures):
            directory = futures[future]
            try:
                return_code = future.result()
                status = "SUCCESS" if return_code == 0 else f"FAILED ({return_code})"
                print(f"{Path(directory).name}: {status}")
            except Exception as e:
                print(f"{Path(directory).name}: EXCEPTION - {e}")
    
    print("All tasks completed.")

Processing 1 directories...
Processing: files
[files] ERROR: 2025-07-10 14:00:35,999 - INFO - Found 8261 PDF files. Processing...

Processing PDFs:   0%|          | 0/8261 [00:00<?, ?it/s]2025-07-10 14:00:44,663 - INFO - Skipping '341739_FA%20SIF%20AR%202014.pdf' as its output file already exists.
2025-07-10 14:00:44,664 - INFO - Skipping '341755_Mermaid_Annual_Report_2014_2_05042015.pdf' as its output file already exists.
2025-07-10 14:00:44,664 - INFO - Skipping '341758_SEGL_Annual_Report_6_April_2015.pdf' as its output file already exists.
2025-07-10 14:00:44,664 - INFO - Skipping '341793_MSE-Annual_Report-2014-FINAL-20150406.pdf' as its output file already exists.

Processing PDFs:   0%|          | 1/8261 [00:06<14:39:16,  6.39s/it]2025-07-10 14:00:44,665 - INFO - Skipping '341799_CL_AR2014.pdf' as its output file already exists.
2025-07-10 14:00:44,665 - INFO - Skipping '341801_Riverstone%20Annual%20Report%202014.pdf' as its output file already exists.
2025-07-10 14:00:44,666 - IN

# 保存截取的PDF，后续再把截取的PDF给LLM（已弃用）

In [ ]:
"""extract_board_pages.py

Scan a directory containing >1,000 SGX‑listed companies' annual‑report PDFs and
produce a CSV file (file_name-company-year-board_pages-is_sgx_ar.csv) with four columns:
    1. file_name   – original PDF filename
    2. company     – inferred company name (taken from filename unless overridden
                     by metadata)
    3. year        – four‑digit report year (inferred from filename / first page)
    4. board_pages – comma‑separated list of zero‑based page indices that belong
                     to a Board‑of‑Directors / Management section, e.g. "12,13,14,32,33"

If a PDF appears _not_ to be an SGX annual report (heuristics can be improved),
the script prints an **ALERT** line so you can review it later.

Usage (Python ≥ 3.9):
    pip install pypdf tqdm python‑dateutil

    python extract_board_pages.py /path/to/pdf/folder

A file named file_name-company-year-board_pages-is_sgx_ar.csv will be created in the working directory.
"""
from __future__ import annotations

import csv
import os
import re
import sys
from pathlib import Path
from typing import List, Optional

from dateutil.parser import parse as date_parse
from pypdf import PdfReader
from tqdm import tqdm

In [ ]:

# ---------------------------------------------------------------------------
# Heuristics & Config
# ---------------------------------------------------------------------------

# Any bookmark (outline title) matching one of these patterns is treated as a
# Board / Management section root.
BKMK_PATTERNS = re.compile(
    r"(board\s+of\s+directors|directors'\s+profile|directors\b|management|key\s+management|executive\s+team)",
    re.IGNORECASE,
)

# Fallback: heading patterns to search in page text when no useful bookmarks
HEADING_PATTERNS = re.compile(
    r"^(?:\s*chapter\s+\d+\s+)?(?:section\s+\d+\s+)?(board\s+of\s+directors|directors'\s+profile|directors\b|management\s+team|key\s+management).*",
    re.IGNORECASE | re.MULTILINE,
)

# Rough heuristic keywords that suggest the PDF is _probably_ an SGX annual
# report. Adjust as needed.
ANNUAL_RPT_KEYWORDS = (
    "annual report",
    "sgx",
    "singapore exchange",
)

# Year pattern (e.g. 2024) for extraction
YEAR_RE = re.compile(r"20\d{2}")

# ---------------------------------------------------------------------------


# 确保 BKMK_PATTERNS 变量已经定义, 例如:
# BKMK_PATTERNS = re.compile(r'board of directors|management|key personnel', re.IGNORECASE)

def extract_board_pages(reader: PdfReader) -> List[int]:
    """
    通过书签识别“董事会”等相关章节的页面范围。
    返回一个排序后的、唯一的、从0开始的页面索引列表。
    """
    all_bookmarks = []

    # 1. 递归函数，用于展平书签树并获取所有书签的标题和页码
    def _flatten_bookmarks(items):
        for it in items:
            if isinstance(it, list):
                _flatten_bookmarks(it)
            else:
                try:
                    # get_destination_page_number 返回从0开始的页码索引
                    page_num = reader.get_destination_page_number(it)
                    title = getattr(it, "title", "")
                    if title:
                        all_bookmarks.append({'title': str(title), 'page': page_num})
                except Exception:
                    # 某些书签可能无效或无法解析，跳过它们
                    pass

    try:
        outlines = reader.outline if hasattr(reader, 'outline') else []
        _flatten_bookmarks(outlines)
    except Exception:
        # 如果没有书签或书签格式错误
        return []

    if not all_bookmarks:
        return []

    # 2. 按页码对所有书签进行排序
    all_bookmarks.sort(key=lambda x: x['page'])
    
    # 去除指向同一页的重复书签，保留第一个
    unique_page_bookmarks = []
    seen_pages = set()
    for bm in all_bookmarks:
        if bm['page'] not in seen_pages:
            unique_page_bookmarks.append(bm)
            seen_pages.add(bm['page'])
    
    all_bookmarks = unique_page_bookmarks

    # 3. 确定每个相关章节的页面范围
    pages_to_extract = set()
    num_bookmarks = len(all_bookmarks)
    total_pages = len(reader.pages)

    for i, bookmark in enumerate(all_bookmarks):
        # 检查书签标题是否匹配我们的关键词
        if BKMK_PATTERNS.search(bookmark['title']):
            start_page = bookmark['page']
            
            # 确定结束页。它是下一个书签的起始页。
            # 如果这是最后一个书签，则章节延伸到PDF的末尾。
            end_page = all_bookmarks[i + 1]['page'] if i + 1 < num_bookmarks else total_pages
            
            # 4. 将范围 [start_page, end_page) 内的所有页面索引添加到集合中
            for page_index in range(start_page, end_page):
                pages_to_extract.add(page_index)

    # 2️⃣ Fallback: scan page text for headings when no bookmarks found
    if pages_to_extract.isEmpty():
        for i, page in enumerate(reader.pages):
            text = "\n".join(page.extract_text().splitlines()[:5])  # top of page
            if HEADING_PATTERNS.search(text):
                pages_to_extract.add(i)

    return sorted(list(pages_to_extract))


def infer_year(name: str, first_page_text: str) -> Optional[str]:
    """Infer a 4‑digit year from filename or first page; return None if not found."""
    # Priority 1: filename walk left‑to‑right
    m = YEAR_RE.search(name)
    if m:
        return m.group(0)
    # Priority 2: parse dates in first page text
    for m in YEAR_RE.finditer(first_page_text[:200]):
        year = m.group(0)
        # discard unlikely years beyond current year +1
        try:
            if 1990 < int(year) < 2100:
                return year
        except ValueError:
            pass
    return None


def is_probably_sgx_ar(first_page_text: str) -> bool:
    text_lower = first_page_text.lower()
    return any(k in text_lower for k in ANNUAL_RPT_KEYWORDS)


# ---------------------------------------------------------------------------
# Main driver
# ---------------------------------------------------------------------------

def process_pdf(path: Path):
    reader = PdfReader(str(path))
    first_page_text = reader.pages[0].extract_text() or ""

    company = path.stem  # default to filename sans extension

    year = infer_year(path.name, first_page_text) or "UNKNOWN"

    board_pages = extract_board_pages(reader)

    is_sgx_ar = is_probably_sgx_ar(first_page_text)

    return {
        "file_name": path.name,
        "company": company,
        "year": year,
        "board_pages": ",".join(map(str, board_pages)) if board_pages else "",
        "is_sgx_ar": is_sgx_ar,
    }


def main(pdf_dir: Path):
    rows = []
    pdf_files = sorted(pdf_dir.glob("*.pdf"))
    if not pdf_files:
        print("No PDF files found in", pdf_dir, file=sys.stderr)
        return

    for pdf in tqdm(pdf_files, desc="Processing PDFs"):
        try:
            info = process_pdf(pdf)
        except Exception as exc:
            print(f"ERROR processing {pdf.name}: {exc}", file=sys.stderr)
            continue
        if not info["is_sgx_ar"]:
            print(f"ALERT: {pdf.name} may not be an SGX annual report")
        rows.append(info)

    # write CSV
    with open("file_name-company-year-board_pages-is_sgx_ar.csv", "w", newline="", encoding="utf-8-sig") as fh:
        writer = csv.DictWriter(
            fh,
            fieldnames=["file_name", "company", "year", "board_pages"],
        )
        writer.writeheader()
        for r in rows:
            writer.writerow({k: r[k] for k in writer.fieldnames})

    print("CSV saved to file_name-company-year-board_pages-is_sgx_ar.csv with", len(rows), "entries")

In [6]:
main(Path("G:\\My Drive\\NUS MSBA SEM2\\<Base Folder>\\files").expanduser().resolve())

Processing PDFs:   0%|          | 4/2519 [00:26<5:47:03,  8.28s/it]

ALERT: 705904_ICTSI%20SEC%20Form%2017A%20and%20Consolidated%20FS%202021.pdf may not be an SGX annual report


Processing PDFs:   0%|          | 7/2519 [01:18<9:32:01, 13.66s/it] 

ALERT: 706479_Trendlines%20-%20Annual%20Report%202021.pdf may not be an SGX annual report


Processing PDFs:   0%|          | 9/2519 [01:44<9:09:11, 13.13s/it] 

...

ALERT: 849072_20250617Mitsuico2503ASR.pdf may not be an SGX annual report


Processing PDFs: 100%|█████████▉| 2518/2519 [9:41:30<00:14, 14.28s/it]

ALERT: 849078_Consolidated%20Financial%20Statements%20NTTFINANCE%20CORPORATION%20AND%20CONSOLIDATED%20SUBSIDIARIES.pdf may not be an SGX annual report


Advanced encoding /90ms-RKSJ-H not implemented yet
Processing PDFs: 100%|██████████| 2519/2519 [9:41:37<00:00, 13.85s/it]

ALERT: 849164_mcpr250618e.pdf may not be an SGX annual report
CSV saved to file_name-company-year-board_pages-is_sgx_ar.csv with 2513 entries


In [11]:
"""extract_people_pages.py

Given:
    * a directory containing SGX annual‑report PDFs, and
    * the CSV file produced by `extract_board_pages.py` (columns: file_name, company, year, board_pages),

create, for every row that has non‑empty `board_pages`, a new PDF containing
**only** those pages.  The output files are written to a subdirectory named
`ONLY_PEOPLE_PAGES` located inside the same folder as the source PDF.

Output filename pattern:
    <original_basename>_keep_only_key_personnel.pdf

Example:
    $ python extract_people_pages.py /path/to/reports report.csv

Dependencies:
    pip install pypdf pandas tqdm  # PyPDF2>=3, renamed as pypdf

Notes:
    * Page indices in `board_pages` are assumed to be zero‑based, matching the
      positions counted from the cover page in the original PDF.
    * If a row is missing or empty `board_pages`, that PDF is skipped.
    * Any issues are logged, and a summary is printed on completion.
"""
from __future__ import annotations

import argparse
import csv
import logging
import sys
from pathlib import Path
from typing import List

import pandas as pd
from pypdf import PdfReader, PdfWriter
from tqdm import tqdm
from pathlib import Path

logging.basicConfig(
    format="[%(levelname)s] %(message)s", level=logging.INFO, stream=sys.stdout
)

In [ ]:
def parse_args() -> argparse.Namespace:
    parser = argparse.ArgumentParser(
        description="Extract specified pages (board / people pages) from each PDF and save as new PDF."
    )
    parser.add_argument(
        "pdf_dir",
        type=Path,
        help="Directory containing the original PDF files (recursively searched)",
    )
    parser.add_argument(
        "csv_path",
        type=Path,
        help="Path to CSV generated by extract_board_pages.py",
    )
    parser.add_argument(
        "--output-subdir",
        default="ONLY_PEOPLE_PAGES",
        help="Name of subdirectory (inside each PDF's parent) for extracted PDFs",
    )
    return parser.parse_args()


def load_tasks(csv_path: Path, pdf_dir: Path) -> List[tuple[Path, List[int]]]:
    df = pd.read_csv(csv_path)
    tasks: List[tuple[Path, List[int]]] = []

    for _, row in df.iterrows():
        fname = str(row["file_name"]).strip()
        pages_str = str(row["board_pages"]).strip()
        if not fname or not pages_str or pages_str.lower() in {"nan", ""}:
            continue  # skip rows without pages

        pdf_path = (pdf_dir / fname).resolve()
        if not pdf_path.exists():
            logging.warning("PDF not found: %s", pdf_path)
            continue

        # Parse comma‑separated list into ints, ignore empty parts
        try:
            pages = [int(p) for p in pages_str.split(",") if p.strip()]
        except ValueError:
            logging.warning(
                "Invalid page list '%s' for file %s ‑ skipping", pages_str, fname
            )
            continue

        tasks.append((pdf_path, pages))
    return tasks


def extract_pages(pdf_path: Path, pages: List[int], output_subdir: str):
    try:
        reader = PdfReader(str(pdf_path))
    except Exception as e:
        logging.error("Failed to read %s: %s", pdf_path, e)
        return False

    writer = PdfWriter()

    max_idx = len(reader.pages) - 1
    for page_idx in pages:
        if 0 <= page_idx <= max_idx:
            writer.add_page(reader.pages[page_idx])
        else:
            logging.warning(
                "Page index %d out of bounds for %s (max %d)",
                page_idx,
                pdf_path.name,
                max_idx,
            )
    if writer.get_num_pages() == 0:
        logging.warning("No valid pages extracted for %s – skipped", pdf_path.name)
        return False

    # Build output path
    out_dir = pdf_path.parent / output_subdir
    out_dir.mkdir(parents=True, exist_ok=True)
    out_name = pdf_path.stem + "_keep_only_key_personnel.pdf"
    out_path = out_dir / out_name

    try:
        with out_path.open("wb") as f:
            writer.write(f)
        return True
    except Exception as e:
        logging.error("Failed to write %s: %s", out_path, e)
        return False

In [17]:
csv_path="G:\\My Drive\\NUS MSBA SEM2\\<Base Folder>\\SGX Annual Reports\\Case Study\\Venture Corporation Limited\\file_name-company-year-board_pages-is_sgx_ar.csv"
pdf_dir="C:\\Users\\22601\\Downloads\\downloads\\files"
output_subdir = pdf_dir+'\\extracted_key_personnel_pages'

tasks = load_tasks(csv_path=csv_path, pdf_dir=pdf_dir)

if not tasks:
    logging.error("No extraction tasks found – check your CSV and directory paths.")
    sys.exit(1)

logging.info("Starting extraction for %d PDFs", len(tasks))
success_cnt = 0

for pdf_path, pages in tqdm(tasks, desc="Extracting", unit="file"):
    ok = extract_pages(Path(pdf_path), pages, output_subdir)
    if ok:
        success_cnt += 1

logging.info(
    "Finished: %d/%d PDFs processed successfully (output in '%s' subdir).",
    success_cnt,
    len(tasks),
    output_subdir,
)

[INFO] Starting extraction for 2241 PDFs


Extracting:  21%|██        | 460/2241 [04:17<17:22,  1.71file/s]

[WARNING] Ignoring wrong pointing object 32 0 (offset 0)
[WARNING] Ignoring wrong pointing object 34 0 (offset 0)
[WARNING] Ignoring wrong pointing object 36 0 (offset 0)
[WARNING] Ignoring wrong pointing object 60 0 (offset 0)
[WARNING] Ignoring wrong pointing object 72 0 (offset 0)
[WARNING] Ignoring wrong pointing object 132 0 (offset 0)
[WARNING] Ignoring wrong pointing object 141 0 (offset 0)
[WARNING] Ignoring wrong pointing object 143 0 (offset 0)
[WARNING] Ignoring wrong pointing object 192 0 (offset 0)
[WARNING] Ignoring wrong pointing object 1478 0 (offset 0)
[WARNING] Ignoring wrong pointing object 1484 0 (offset 0)
[WARNING] Ignoring wrong pointing object 1494 0 (offset 0)
[WARNING] Ignoring wrong pointing object 1637 0 (offset 0)
[WARNING] Ignoring wrong pointing object 1943 0 (offset 0)


Extracting:  58%|█████▊    | 1303/2241 [13:23<12:19,  1.27file/s]

[WARNING] Ignoring wrong pointing object 6 0 (offset 0)
[WARNING] Ignoring wrong pointing object 8 0 (offset 0)
[WARNING] Ignoring wrong pointing object 16 0 (offset 0)
[WARNING] Ignoring wrong pointing object 19 0 (offset 0)
[WARNING] Ignoring wrong pointing object 21 0 (offset 0)
[WARNING] Ignoring wrong pointing object 58 0 (offset 0)
[WARNING] Ignoring wrong pointing object 199 0 (offset 0)
[WARNING] Ignoring wrong pointing object 219 0 (offset 0)
[WARNING] Ignoring wrong pointing object 221 0 (offset 0)
[WARNING] Ignoring wrong pointing object 225 0 (offset 0)
[WARNING] Ignoring wrong pointing object 227 0 (offset 0)
[WARNING] Ignoring wrong pointing object 229 0 (offset 0)
[WARNING] Ignoring wrong pointing object 236 0 (offset 0)
[WARNING] Ignoring wrong pointing object 279 0 (offset 0)
[WARNING] Ignoring wrong pointing object 281 0 (offset 0)
[WARNING] Ignoring wrong pointing object 283 0 (offset 0)
[WARNING] Ignoring wrong pointing object 285 0 (offset 0)
[WARNING] Ignoring wro

...

[WARNING] Ignoring wrong pointing object 6 0 (offset 0)
[WARNING] Ignoring wrong pointing object 127 0 (offset 0)
[WARNING] Ignoring wrong pointing object 129 0 (offset 0)
[WARNING] Ignoring wrong pointing object 131 0 (offset 0)
[WARNING] Ignoring wrong pointing object 133 0 (offset 0)
[WARNING] Ignoring wrong pointing object 135 0 (offset 0)
[WARNING] Ignoring wrong pointing object 137 0 (offset 0)
[WARNING] Ignoring wrong pointing object 139 0 (offset 0)
[WARNING] Ignoring wrong pointing object 141 0 (offset 0)
[WARNING] Ignoring wrong pointing object 904 0 (offset 0)
[WARNING] Ignoring wrong pointing object 910 0 (offset 0)
[WARNING] Ignoring wrong pointing object 912 0 (offset 0)
[WARNING] Ignoring wrong pointing object 914 0 (offset 0)
[WARNING] Ignoring wrong pointing object 916 0 (offset 0)
[WARNING] Ignoring wrong pointing object 918 0 (offset 0)
[WARNING] Ignoring wrong pointing object 1297 0 (offset 0)
[WARNING] Ignoring wrong pointing object 1363 0 (offset 0)


Extracting:  81%|████████▏ | 1826/2241 [19:13<04:14,  1.63file/s]

[WARNING] Ignoring wrong pointing object 100 0 (offset 0)
[WARNING] Ignoring wrong pointing object 699 0 (offset 0)
[WARNING] Ignoring wrong pointing object 1130 0 (offset 0)
[WARNING] Ignoring wrong pointing object 1193 0 (offset 0)
[WARNING] Ignoring wrong pointing object 1194 0 (offset 0)
[WARNING] Ignoring wrong pointing object 1195 0 (offset 0)
[WARNING] Ignoring wrong pointing object 1196 0 (offset 0)
[WARNING] Ignoring wrong pointing object 1197 0 (offset 0)
[WARNING] Ignoring wrong pointing object 1198 0 (offset 0)
[WARNING] Ignoring wrong pointing object 1199 0 (offset 0)
[WARNING] Ignoring wrong pointing object 1200 0 (offset 0)
[WARNING] Ignoring wrong pointing object 1201 0 (offset 0)
[WARNING] Ignoring wrong pointing object 1202 0 (offset 0)
[WARNING] Ignoring wrong pointing object 1203 0 (offset 0)
[WARNING] Ignoring wrong pointing object 1204 0 (offset 0)
[WARNING] Ignoring wrong pointing object 1205 0 (offset 0)
[WARNING] Ignoring wrong pointing object 1206 0 (offset 0)

Extracting: 100%|██████████| 2241/2241 [23:59<00:00,  1.56file/s]

[INFO] Finished: 2241/2241 PDFs processed successfully (output in 'C:\Users\22601\Downloads\downloads\files\extracted_key_personnel_pages' subdir).
